# Note: The following file will fail if you will try to run it as it must contain private Spotify client settings.

First we had to install spotipy library by running `pip install spotipy`
<br>
According to https://anaconda.org/jkroes/spotipy

#### Using Spotify API request to approve few terms of use:<br>
* I understand that this app is not for commercial use
* I understand that I cannot migrate my app from non-commercial to commercial without permission
* I understand and agree with Spotify's Developer Terms of Service, Branding Guidelines, and Privacy Policy
    * https://developer.spotify.com/terms/
    * https://developer.spotify.com/branding-guidelines/
    * https://www.spotify.com/il-en/legal/privacy-policy/

Create access token and get a specific user liked songs (as a test)

In [1]:
import spotipy
import spotipy.util as util
import sys

username = ''
client_id = ''
client_secret = ''
redirect_uri = 'http://localhost:8888/callback/'
scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_saved_tracks()
    for item in results['items']:
        track = item['track']
        print(track['name'] + ' - ' + track['artists'][0]['name'])
else:
    print("Can't get token for", username)

Backseat Freestyle - Kendrick Lamar
Tusa - KAROL G
Ready To Let Go - Cage The Elephant
Diggin' a Hole - Downstairs Monsters
Starry Night - Original Mix - Peggy Gou
That's Life (feat. Mac Miller & Sia) - 88-Keys
Eleven - Khalid
Lalala - Y2K
מאושרים - Doli & Penn
ROXANNE - Arizona Zervas
Be Still - Liam Gallagher
לא חסר לי כלום - Avihu Pinchasov Rhythm Club
Sunday Best - Surfaces
Right Back (feat. A Boogie Wit Da Hoodie) - Khalid
I Got A Name - Stereo Version - Jim Croce
Lover Of The Light - Live From Red Rocks, Colorado - Mumford & Sons
What's the Use? - Mac Miller
Hard Sun - Eddie Vedder
The Girl from Ipanema - Al Hirt
The Unexpected Visitor - CBL Carbonator Remix - Shulman


In [2]:
track_id = []


if token:
    sp = spotipy.Spotify(auth=token)
    playlists = sp.user_playlists(username)
    print(playlists)
else:
    print("Can't get token for", username)

{'href': 'https://api.spotify.com/v1/users/-f/playlists?offset=0&limit=50', 'items': [], 'limit': 50, 'next': None, 'offset': 0, 'previous': None, 'total': 0}


We had the option to include 'popularity' column but as it's an updated data to __todays__ popularity status of the song then it's not relevant - we marked those lines.

In [7]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
from pprint import pprint

pl2015_uri = 'spotify:playlist:1xkuRzFhnX3hCWu6hLxJ0U' #contains 95
pl2016_uri = 'spotify:playlist:2LWafCgWzsXGWv7wJeePjA' #contains 98
pl2017_uri = 'spotify:playlist:3CWVCLSmoT7speb9E5b9st' #contains 100
pl2018_uri = 'spotify:playlist:37IRJrV9jd0LnsFTIY83ax' #contains 100

ids2015_list = []
ids2016_list = []
ids2017_list = []
ids2018_list = []

offset = 0
i = 0
while True:
    ids = sp.playlist_tracks(pl2018_uri,
                                  offset=offset,
                                  fields='items.track.id,total')
#     pops = sp.playlist_tracks(pl2018_uri,
#                                   offset=offset,
#                                   fields='items.track.popularity.total')
    offset = offset + len(ids['items'])
    print(offset)
    print(len(ids['items']))
    #print(offset, "/", response['total'])
    
    for i in range(0, offset - 1):
        print(ids['items'][i])
        print(i)
        #pprint(ids['items'][i].get('track').get('id'))
        #ids2018_list.append(response['items'][i].get('track').get('id'))
        
        #pprint(pops.get('items')[i].get('track').get('popularity'))
    if len(ids['items']) == 0:
        break
#ids2018_list

100
100
{'track': {'id': '6DCZcSspjsKoFjzjrWoCdn'}}
0
{'track': {'id': '0tgVpDi06FyKpA1z0VMD4v'}}
1
{'track': {'id': '7iDa6hUg2VgEL1o1HjmfBn'}}
2
{'track': {'id': '1rfofaqEpACxVEHIZBJe6W'}}
3
{'track': {'id': '0e7ipj03S05BNilyu5bRzt'}}
4
{'track': {'id': '3swc6WTsr7rl9DqQKQA55C'}}
5
{'track': {'id': '58q2HKrzhC3ozto2nDdN4z'}}
6
{'track': {'id': '09IStsImFySgyp0pIQdqAc'}}
7
{'track': {'id': '2G7V7zsVDxg1yRsu7Ew9RJ'}}
8
{'track': {'id': '6FRLCMO5TUHTexlWo8ym1W'}}
9
{'track': {'id': '3CA9pLiwRIGtUBiMjbZmRw'}}
10
{'track': {'id': '0s3nnoMeVWz3989MkNQiRf'}}
11
{'track': {'id': '7dt6x5M1jzdTEt8oCbisTK'}}
12
{'track': {'id': '3Vo4wInECJQuz9BIBMOu8i'}}
13
{'track': {'id': '0At2qAoaVjIwWNAqrscXli'}}
14
{'track': {'id': '6tF92PMv01Ug9Dh8Rmy6nH'}}
15
{'track': {'id': '3ee8Jmje8o58CHK66QrVC2'}}
16
{'track': {'id': '4eWQlBRaTjPPUlzacqEeoQ'}}
17
{'track': {'id': '45Egmo7icyopuzJN0oMEdk'}}
18
{'track': {'id': '2qT1uLXPVPzGgFOx4jtEuo'}}
19
{'track': {'id': '7uzmGiiJyRfuViKKK3lVmR'}}
20
{'track': {'id'

IndexError: list index out of range

In [204]:
feature_results = sp.audio_features('0pqnGHJpmpxLKifKRmU6WP')
feature_results

[{'danceability': 0.776,
  'energy': 0.78,
  'key': 10,
  'loudness': -4.374,
  'mode': 0,
  'speechiness': 0.128,
  'acousticness': 0.0622,
  'instrumentalness': 0,
  'liveness': 0.081,
  'valence': 0.666,
  'tempo': 124.949,
  'type': 'audio_features',
  'id': '0pqnGHJpmpxLKifKRmU6WP',
  'uri': 'spotify:track:0pqnGHJpmpxLKifKRmU6WP',
  'track_href': 'https://api.spotify.com/v1/tracks/0pqnGHJpmpxLKifKRmU6WP',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0pqnGHJpmpxLKifKRmU6WP',
  'duration_ms': 204347,
  'time_signature': 4}]

In [213]:
import pandas as pd
temp_ids = ['0pqnGHJpmpxLKifKRmU6WP', '4gB7HrYHbJVJ5RFOjxmoq4', '76cy1WJvNGJTj78UqeA5zr']

temp_df = pd.DataFrame()
temp_df['ids'] = temp_ids
print(temp_df, '\n')

                      ids
0  0pqnGHJpmpxLKifKRmU6WP
1  4gB7HrYHbJVJ5RFOjxmoq4
2  76cy1WJvNGJTj78UqeA5zr 



In [214]:
# again measuring the time
import timeit
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(temp_df['ids']),batchsize):
    batch = temp_df['ids'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)

print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 0.10332779999953345


In [202]:
print('number of elements in the track_id list:', len(rows))
print('\n', rows[0])

number of elements in the track_id list: 3

 {'danceability': 0.776, 'energy': 0.78, 'key': 10, 'loudness': -4.374, 'mode': 0, 'speechiness': 0.128, 'acousticness': 0.0622, 'instrumentalness': 0, 'liveness': 0.081, 'valence': 0.666, 'tempo': 124.949, 'type': 'audio_features', 'id': '0pqnGHJpmpxLKifKRmU6WP', 'uri': 'spotify:track:0pqnGHJpmpxLKifKRmU6WP', 'track_href': 'https://api.spotify.com/v1/tracks/0pqnGHJpmpxLKifKRmU6WP', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0pqnGHJpmpxLKifKRmU6WP', 'duration_ms': 204347, 'time_signature': 4}


Load the audio features data into a dataframe

In [203]:
df_audio_features.head()
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)

Shape of the dataset: (3, 18)


,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.06220,https://api.spotify.com/v1/audio-analysis/0pqn...,0.776,204347,0.780,0pqnGHJpmpxLKifKRmU6WP,0,10,0.0810,-4.374,0,0.1280,124.949,4,https://api.spotify.com/v1/tracks/0pqnGHJpmpxL...,audio_features,spotify:track:0pqnGHJpmpxLKifKRmU6WP,0.666
1,0.00585,https://api.spotify.com/v1/audio-analysis/4gB7...,0.544,221800,0.781,4gB7HrYHbJVJ5RFOjxmoq4,0,6,0.1910,-6.160,0,0.0311,144.075,4,https://api.spotify.com/v1/tracks/4gB7HrYHbJVJ...,audio_features,spotify:track:4gB7HrYHbJVJ5RFOjxmoq4,0.527
2,0.04030,https://api.spotify.com/v1/audio-analysis/76cy...,0.836,217947,0.544,76cy1WJvNGJTj78UqeA5zr,0,7,0.0824,-5.975,1,0.0943,97.028,4,https://api.spotify.com/v1/tracks/76cy1WJvNGJT...,audio_features,spotify:track:76cy1WJvNGJTj78UqeA5zr,0.510


In [222]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.dtypes

Shape of the dataset: (3, 18)


acousticness        float64
analysis_url         object
danceability        float64
duration_ms           int64
energy              float64
id                   object
instrumentalness      int64
key                   int64
liveness            float64
loudness            float64
mode                  int64
speechiness         float64
tempo               float64
time_signature        int64
track_href           object
type                 object
uri                  object
valence             float64
dtype: object

Processing the data - drop uneeded columns

In [223]:
df_audio_features.drop(['analysis_url', 'track_href', 'type', 'uri'], axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.dtypes

Shape of the dataset: (3, 14)


acousticness        float64
danceability        float64
duration_ms           int64
energy              float64
track_id             object
instrumentalness      int64
key                   int64
liveness            float64
loudness            float64
mode                  int64
speechiness         float64
tempo               float64
time_signature        int64
valence             float64
dtype: object

Merge between audio features dataframe and our original dataframe

In [ ]:
# the 'inner' method will make sure that we only keep track IDs present in both datasets
# df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
# print("Shape of the dataset:", df_audio_features.shape)
# df.head()

Cheching if we have any duplication in track

In [224]:
#df[df.duplicated(subset=['artist_name','track_name'],keep=False)]

Saving the new dataframe as csv for outer use

In [ ]:
#df.to_csv('myname.csv')